In [2]:
organism = "org.At.tair.db"


In [ ]:
BiocManager::install(organism, character.only = TRUE)

In [100]:

#conda install -c bioconda bioconductor-pathview 
#BiocManager::install("enrichplot")

Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.3 (2020-02-29)

Installing package(s) 'pathview'

also installing the dependency ‘KEGGgraph’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'backports', 'BiocParallel', 'clusterProfiler', 'data.table',
  'GOSemSim', 'IRanges', 'S4Vectors'



In [99]:
library(clusterProfiler)
library(tidyverse)
library(pathview)
#library(enrichplot)
#use organism org.At.tair.db (arabidopsis) - follow https://learn.gencore.bio.nyu.edu/rna-seq-analysis/over-representation-analysis/
library(organism, character.only = TRUE)

ERROR: Error in library(pathview): there is no package called ‘pathview’


In [11]:
#search for arabidopsis
search_kegg_organism('ath', by='kegg_code')

,kegg_code,scientific_name,common_name
,<chr>,<chr>,<chr>
133,ath,Arabidopsis thaliana,thale cress


In [6]:
genes_of_interest = '../../data/output/non-overlapping_includingbidirectional_all_genes_newannotation/genes/promoters_5UTR_czechowski_constitutive_variable_random_300.txt'
background_gene_set = '../../data/output/non-overlapping_includingbidirectional_all_genes_newannotation/genes/gene_ontology/gene_result_filtered.txt'
mapped_NCBI_geneset = '../../data/output/non-overlapping_includingbidirectional_all_genes_newannotation/genes/gene_ontology/mapped_NCBI.tsv'

In [105]:
#KEGG Gene Set Enrichment Analysis
KEGG_GSEA = function(genes_of_interest,background_gene_set,gene_type) {
    ###function to run a KEGG over-representation test using genes of interest, backgroun file and specifying the gene_type category###
    genes_of_interest_df = read.delim(genes_of_interest, sep='\t',fill = TRUE,header=TRUE)
    background_gene_set_df = read.delim(background_gene_set,sep='\t',header=TRUE)
    
    #make background gene table columns characters
    background_gene_set_df = data.frame(lapply(background_gene_set_df, as.character), stringsAsFactors=FALSE)

    #remove nan from background genes
    background_genes = na.omit(background_gene_set_df$GeneID)

    #convert NCBI gene IDs to AGIs to ensure they are in the database
    IDs = bitr(background_genes, fromType='ENTREZID', toType='TAIR',OrgDb=organism)
    
    #remove duplicates
    background_dedup_ids = IDs[!duplicated(IDs[c("ENTREZID")]),]
    
    #sort the IDs in descending order
    sorted_background = sort(background_dedup_ids$ENTREZID,decreasing = TRUE)
    
    #remove NAN
    sorted_background = na.omit(sorted_background)
    
    #select columns from genes_of_interest_df
    #NCBI_genes = genes_of_interest_df %>% select('GeneID','AGI','gene_type')
    #return(NCBI_genes)
    #filter by gene type
    genes = filter(genes_of_interest_df, gene_type == gene_type)
    
    #turn columns into characters/string
    #genes = data.frame(lapply(genes, as.character), stringsAsFactors=FALSE)
    #return(genes)
    
    #turn the gene list into a vector
    avector = genes$AGI

    #remove NAN
    avector = na.omit(avector)

    #convert NCBI gene IDs to kegg IDs
    #IDs = bitr_kegg(geneID = avector, fromType='ncbi-geneid', toType='kegg', organism='ath')
    IDs = bitr(avector, fromType='TAIR', toType='ENTREZID',OrgDb=organism)
    #remove duplicates
    dedup_ids = IDs[!duplicated(IDs[c("TAIR")]),]

    
    #sort the IDs in descending order
    sorted_genes_vector = sort(dedup_ids$ENTREZID,decreasing = TRUE)
    #remove NAN
    genes_list = na.omit(sorted_genes_vector)

    #return(sorted_genes_vector)
    #return(IDs)
    #sort the IDs
    #IDs_sorted = IDs[order('kegg'),]
    #return(IDs_sorted)
    #return(IDs_sorted)
    #return(avector_sorted_characters)
    #return(avector_sorted)
    #background_vector = as.vector(background_gene_set_df['GeneID'])
    #KEGG over-representation test - hochberg p adjust
    enrichment = enrichKEGG(gene=genes_list,organism = 'ath', pAdjustMethod='BH',pvalueCutoff=0.05,keyType='ncbi-geneid',minGSSize = 1,maxGSSize = 900,universe=sorted_background)
    return(list(enrichment,genes_list))
}

In [106]:
output = KEGG_GSEA(mapped_NCBI_geneset,background_gene_set,gene_type='constitutive')
constitutive_enrichment=output[1]
constitutive_genes=output[2]

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(background_genes, fromType = "ENTREZID", toType = "TAIR", :
“0.01% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns



In [107]:
constitutive_enrichment

[[1]]
#
# over-representation test
#
#...@organism 	 ath 
#...@ontology 	 KEGG 
#...@keytype 	 ncbi-geneid 
#...@gene 	 chr [1:900] "844389" "844345" "844339" "844334" "844303" "844250" "844216" ...
#...pvalues adjusted by 'BH' with cutoff <0.05 
#...0 enriched terms found
#...Citation
  Guangchuang Yu, Li-Gen Wang, Yanyan Han and Qing-Yu He.
  clusterProfiler: an R package for comparing biological themes among
  gene clusters. OMICS: A Journal of Integrative Biology
  2012, 16(5):284-287 



In [ ]:
#no enriched constitutive genes

In [108]:
output2 = KEGG_GSEA(mapped_NCBI_geneset,background_gene_set,gene_type='variable')
variable_enrichment=output2[1]
variable_genes=output2[2]

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(background_genes, fromType = "ENTREZID", toType = "TAIR", :
“0.01% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns



In [109]:
variable_enrichment

[[1]]
#
# over-representation test
#
#...@organism 	 ath 
#...@ontology 	 KEGG 
#...@keytype 	 ncbi-geneid 
#...@gene 	 chr [1:900] "844389" "844345" "844339" "844334" "844303" "844250" "844216" ...
#...pvalues adjusted by 'BH' with cutoff <0.05 
#...0 enriched terms found
#...Citation
  Guangchuang Yu, Li-Gen Wang, Yanyan Han and Qing-Yu He.
  clusterProfiler: an R package for comparing biological themes among
  gene clusters. OMICS: A Journal of Integrative Biology
  2012, 16(5):284-287 



In [ ]:
# # Produce the native KEGG plot (PNG)
# dme <- pathview(gene.data=constitutive_genes, pathway.id="dme04080", species = kegg_organism, gene.idtype=gene.idtype.list[3])

# # Produce a different plot (PDF) (not displayed here)
# dme <- pathview(gene.data=gene_list, pathway.id="dme04080", species = kegg_organism, gene.idtype=gene.idtype.list[3], kegg.native = F)

# knitr::include_graphics("dme04080.pathview.png")